In [26]:
import plotly.graph_objects as go
import numpy as np
from numpy import *
import math
import plotly.express as px
from decimal import *

In [27]:
def Vec2UnitVec(vec):
    norm = np.sqrt( np.square(vec[0]) + np.square(vec[1]) + np.square(vec[2]))
    vector = vec / norm
    return vector

def FindIntersectionPoints(x1,y1,z1,x2,y2,z2,T):
    PointX = x1 + T* (x2 - x1)
    PointY = y1 + T* (y2 - y1)
    PointZ = z1 + T* (z2 - z1)
    return [PointX,PointY,PointZ] 

def AnglebtwnVecs(vec1,vec2):
    angle = np.arccos(np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))
#     print("Angle : {}".format((angle*180)/math.pi))
    return angle

def EuclideanDistance(vec1,vec2):
    E2Distance = np.sqrt( np.square(vec1[0]- vec2[0]) + np.square(vec1[1]- vec2[1]) + np.square(vec1[2]- vec2[2]))
    return E2Distance

def ComputeTransformation(xpoints,ypoints,zpoints,TMat,flag):
    Xp = []
    Yp = []
    Zp = []

    for i in range(len(xpoints)):
        vect = [xpoints[i],ypoints[i],zpoints[i]]
        if flag == 1:
            TransformedCodinates1 = np.dot(TMat[0],vect)
            TransformedCodinates2 = np.dot(TMat[1],TransformedCodinates1)
        else : 
            TransformedCodinates2 = np.dot(TMat,vect)
            
        Xp.append(TransformedCodinates2[0])
        Yp.append(TransformedCodinates2[1])
        Zp.append(TransformedCodinates2[2])
    
    xp = np.reshape(Xp,(100,100))
    yp = np.reshape(Yp,(100,100))
    zp = np.reshape(Zp,(100,100))
    return xp,yp,zp

def CheckInisideorOutsideEllipsoid(p1,mp,Trans):

    vec = np.asarray(p1) - np.asarray(mp)
    result = np.dot(np.dot(vec,Trans),vec)
    return result


In [42]:
#Visulaization:
def VisulaizingEllisoid(fig,c,r1,r2,TMat,Rot,EP1,EP2,Cl1,Cl2,p1,p2):
    D = 10
    theta = linspace(0,2*np.pi,100)
    phi = linspace(0,np.pi,100)
    x =  (r1) * outer(cos(theta),sin(phi))
    y =  (r2) * outer(sin(theta),sin(phi))
    z =  (r2) * outer(ones(100),cos(phi))
    x1,y1,z1 = ComputeTransformation(x.flatten(),y.flatten(),z.flatten(),TMat,1)
    x2,y2,z2 = ComputeTransformation(x.flatten(),y.flatten(),z.flatten(),Rot,0)
    
    x1 = c[0] + x1
    y1 = c[1] + y1
    z1 = c[2] + z1
    
    x2 = c[0] + x2
    y2 = c[1] + y2
    z2 = c[2] + z2
    
    data1=go.Surface(
        x=x1,
        y=y1,
        z=z1,
        opacity=0.7)

    fig.add_trace(data1)

    data2=go.Surface(
        x=x2,
        y=y2,
        z=z2,
        opacity=0.35)

    fig.add_trace(data2)
    
    EllipsoidPoints = go.Scatter3d(x=[EP1[0],EP2[0]], y=[EP1[1],EP2[1]], z=[EP1[2],EP2[2]],
                        line = dict(width=1))
    fig.add_trace(EllipsoidPoints)
    
    CollisionLinePoints = go.Scatter3d(x=[Cl1[0],Cl2[0]], y=[Cl1[1],Cl2[1]], z=[Cl1[2],Cl2[2]],
                        line = dict(width=2))
    fig.add_trace(CollisionLinePoints)
    
    IntersectionPoints = go.Scatter3d(x=[p1[0],p2[0]], y=[p1[1],p2[1]], z=[p1[2],p2[2]],
                        line = dict(width=10))
    fig.add_trace(IntersectionPoints)
    
    fig.update_layout(scene=dict(zaxis=dict(range=[-D,D],autorange=False),       
                    yaxis=dict(range=[-D,D],autorange=False),
                xaxis=dict(range=[-D,D],autorange=False)))  
    return fig

In [28]:
def EllipsoidDetails(Entry,Target):
    
    ModifiedVector = [Entry[0],Entry[1],Target[2]]
    MidPoint = [float(Decimal(Entry[0] + Target[0])/Decimal(2)),float(Decimal(Entry[1] + Target[1])/Decimal(2)),float(Decimal(Entry[2] + Target[2])/Decimal(2))]
   
    MajorAxisRadius = EuclideanDistance(Entry,MidPoint)
    MinorAxisRadius = MajorAxisRadius / 2
   
    DVector1 = np.asarray(Entry) - np.asarray(Target)
    DVector2 = np.asarray(ModifiedVector) - np.asarray(Target)
    
    Basis_X = [1,0,0]
    Basis_Z = [0,0,1]
    
    the1 = AnglebtwnVecs(DVector1,Basis_Z)
    the2 = AnglebtwnVecs(DVector2,Basis_X)
    
    theta1 = math.pi/2 - the1
    
    if Entry[1] > Target[1]:
        theta2 = -the2
    else:
        theta2 = the2
        
    T1 = np.array([[np.cos(theta1),0,-np.sin(theta1)],[0,1,0],[np.sin(theta1),0,np.cos(theta1)]])
    T2 = np.array([[np.cos(theta2),np.sin(theta2),0],[-np.sin(theta2),np.cos(theta2),0],[0,0,1]])
    TMat = [T1,T2]
    
    return MajorAxisRadius,MinorAxisRadius,TMat,MidPoint,theta1,theta2

def CheckEllipsoidCollision(CLpoint1,CLpoint2,MidPoint,MajorAxisRadius,MinorAxisRadius,the1,the2):
   
    stack = []
    #Components:
    l = (np.asarray(CLpoint2) - np.asarray(CLpoint1))
    p = np.asarray(CLpoint1)- np.asarray(MidPoint)
    CoeffMat = np.array([[1/np.square(MajorAxisRadius),0,0],[0,1/np.square(MinorAxisRadius),0],[0,0,1/np.square(MinorAxisRadius)]])
   
    T1 = np.array([[np.cos(the1),0,-np.sin(the1)],[0,1,0],[np.sin(the1),0,np.cos(the1)]])
    T2 = np.array([[np.cos(the2),np.sin(the2),0],[-np.sin(the2),np.cos(the2),0],[0,0,1]])
    
    R = np.dot(T2,T1)
    S = np.dot(np.dot(R,CoeffMat),np.transpose(R))
#     S = np.dot(np.dot(np.transpose(R),CoeffMat),R)

    #Quadratic Constants: 
    a = np.dot(np.dot(l,S),l)
    b = 2 * ( np.dot(np.dot(p,S),l))
    c = (np.dot(np.dot(p,S),p)) - 1
    
    #Solutions
    Solution1 = (-b + np.sqrt(np.square(b) - (4 * a * c))) / (2*a)
    Solution2 = (-b - np.sqrt(np.square(b) - (4 * a * c))) / (2*a)
    
    InOut1 = CheckInisideorOutsideEllipsoid(CLpoint1,MidPoint,S)
    InOut2 = CheckInisideorOutsideEllipsoid(CLpoint2,MidPoint,S)
    
    if np.isnan(Solution1) and np.isnan(Solution2):
        stack.append(-1)
    else:
        IntersectionPoint1 = FindIntersectionPoints(CLpoint1[0],CLpoint1[1],CLpoint1[2],CLpoint2[0],CLpoint2[1],CLpoint2[2],Solution1)
        IntersectionPoint2 = FindIntersectionPoints(CLpoint1[0],CLpoint1[1],CLpoint1[2],CLpoint2[0],CLpoint2[1],CLpoint2[2],Solution2)

        VectorDistance =  EuclideanDistance(CLpoint1,CLpoint2)

        CheckDistance1 = EuclideanDistance(CLpoint1,IntersectionPoint1)
        CheckDistance2 = EuclideanDistance(CLpoint2,IntersectionPoint1)
        CheckDistance3 = EuclideanDistance(CLpoint1,IntersectionPoint2)
        CheckDistance4 = EuclideanDistance(CLpoint2,IntersectionPoint2)

        if CheckDistance1 <= VectorDistance and CheckDistance2 <= VectorDistance:
            stack.append(0)
        elif CheckDistance3 <=VectorDistance and CheckDistance4 <= VectorDistance:
            stack.append(0)
        else:
            stack.append(1)
            MinDistance = [CheckDistance1,CheckDistance2,CheckDistance3,CheckDistance4]
            stack.append(np.min(MinDistance))    
            print("IntersectionPoint1 : {}, IntersectionPoint2 : {}".format(IntersectionPoint1,IntersectionPoint2))
    
    if InOut1 <= 1 or InOut2 <=1:
        stack = []
        stack.append(0)
    
    return IntersectionPoint1,IntersectionPoint2,R,stack

#Main Run 

In [56]:
#Entry Target for ellopsoid (Application) -/+200mm, (Entry[3]>Target[3])
Ellipsoidsample1 = [5, 1, 6]
Ellipsoidsample2 = [1, 2, 1]


#Collision Line points (2 points for line) (Points can be randomized)
CollisionSample1 = [-5,-6,-7]
CollisionSample2 = [4,0,6]

r1,r2,TransMat,MP,the1,the2 = EllipsoidDetails(Ellipsoidsample1,Ellipsoidsample2)
I1,I2,Rot,result = CheckEllipsoidCollision(CollisionSample1,CollisionSample2,MP,r1,r2,the1,the2)

if result == [0]:
    print('Link is Colliding')
elif result ==[-1]:
    print("No collision, Free to go")
else:
    print("No collision, Near object is in distance of {}mm".format(result[1]))

fig = go.Figure()
VisulaizingEllisoid(fig,MP,r1,r2,TransMat,Rot,Ellipsoidsample1,Ellipsoidsample2,CollisionSample1,CollisionSample2,I1,I2)

/home/shivendra/.local/lib/python2.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning:

invalid value encountered in sqrt

/home/shivendra/.local/lib/python2.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning:

invalid value encountered in sqrt



UnboundLocalError: local variable 'IntersectionPoint1' referenced before assignment

In [6]:
print([int(np.random.randint(-200,200,1)),int(np.random.randint(-200,200,1)),int(np.random.randint(21,200,1))])
print([int(np.random.randint(-200,200,1)),int(np.random.randint(-200,200,1)),int(np.random.randint(-200,200,1))])

[-51, -43, 133]
[-195, 26, -100]
